# NEXRAD AWS QVP Code

This code downloads all radar data for a given set of WSR-88D radars between a start and end time, saves them to a chosen directory, and creates QVPs of reflectivity, cross-correlation coefficient, differential phase, and differential reflectivity as well as the vertical gradients and azimuthal variance of each.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import six
import nexradaws
import shutil
from datetime import datetime 
import contextlib
import matplotlib as mpl 
import matplotlib.pyplot as plt
import matplotlib.dates as md
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyart
from metpy.plots import USCOUNTIES
import os 
import numpy as np
import xarray 
import pandas as pd
import logging
from imp import reload
import traceback
from collections import Counter
import pickle
from statsmodels import robust

mpl.rcParams['figure.facecolor'] = 'white'

%config InlineBackend.figure_format='retina' #Make figures high definition so they can be dragged into powerpoint without  saving them

START = datetime(2020,2,25,18,0)
END = datetime(2020,2,26,6,0)
EVENT_DATE = ' ' + str(START)[:-9]

RELEVANT_RADAR_IDS = ['KILX','KLOT','KDVN'] #change this to the radars of interest
PATH = "NEXRAD Scans/" #the highest level directory for saving the files, logs, and figures
DESIRED_ANGLES = [2,3,3.5,4,5,6.5,8,10,12,14,16,19] #Change this to the angle(s) of interest
THRESHOLD = .7 #Minimum Percentage of scans over the interval that contain an angle of interest. If less than this, no QVP will be generated
RELATIVE_ERROR_TOLERANCE = .25 #Maximum relative error between desired angle and the closest angle in the dataset. No QVP will be generated if relative error is greater. Set this higher to generate more QVPs

#Radars in the Northeast, United States
NE_Radars = [['KCLE', 'inland', 'Cleaveland', 'OH'],
             ['KRLX', 'inland', 'Charleston', 'WV'],
             ['KFCX', 'inland', 'Blacksburg', 'VA'],
             ['KLWX', 'inland', 'Sterling', 'VA'],
             ['KAKQ', 'coastal', 'Wakefield', 'VA'],
             ['KDOX', 'coastal', 'Dover', 'DE'],
             ['KDIX', 'coastal', 'Philadelphia', 'PA'],
             ['KPBZ', 'inland', 'Pittsburgh', 'PA'],
             ['KCCX', 'inland', 'State College', 'PA'],
             ['KBGM', 'inland', 'Binghamton', 'NY'],
             ['KBUF', 'inland', 'Buffalo', 'NY'],
             ['KTYX', 'inland', 'Montague', 'NY'],
             ['KENX', 'inland', 'Albany', 'NY'],
             ['KOKX', 'coastal', 'New York City', 'NY'],
             ['KBOX', 'coastal', 'Boston', 'MA'],
             ['KCXX', 'inland', 'Burlington', 'VT'],
             ['KGYX', 'coastal', 'Portland', 'ME'],
             ['KCBW', 'inland', 'Caribou', 'ME']]

In [ ]:
#makes a map of all radar locations available to us througout the northeast

locs = pyart.io.nexrad_common.NEXRAD_LOCATIONS
fig = plt.figure(figsize=(15, 12))
proj = ccrs.Miller()
ax = fig.add_subplot(1, 1, 1, projection=proj)
extent = [-83, -65, 36, 48]
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.coastlines(resolution='10m', color='black', linewidth=0.5)
states_provinces = cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lakes', scale='10m', facecolor='none')
roads = cfeature.NaturalEarthFeature(category='cultural', name='roads', scale='10m', facecolor='none')
ax.add_feature(states_provinces, edgecolor='black', linewidth=0.5)
ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor='black', linewidth=0.2)
ax.add_feature(roads, edgecolor='blue', linewidth=0.3)
plt.title('NEXRAD Radar Locations - Northeast', fontweight='bold')

for key in locs:
    
    lon = locs[key]['lon']
    lat = locs[key]['lat']
    name = key
    ax.scatter(lon,lat,marker='o', color='black', transform=ccrs.PlateCarree())
    lat_mod = lat + .2
    lon_mod = lon - .4

    if lon >= extent[0] and lon <= extent[1] and lat >= extent[2] and lat <= extent[3]:
        ax.text(lon_mod, lat_mod, name, color='black', fontsize=14, transform=ccrs.PlateCarree())

txt = 'University of Illinois'
suppress = ax.text(.83, 0.105, txt, wrap=True, horizontalalignment='center', fontsize=12, fontweight='bold',transform=fig.transFigure)

Data we download employs nine Volume Coverage Patterns (VCPs) to adequately sample the atmosphere based on weather conditions.
A VCP is a series of 360 degree sweeps of the antenna at pre-determined elevation angles and pulse repetition frequencies completed in a specified period of time.
The radar scan times 4.5, 5, 6 or 10 minutesdepending on the selected VCP. The NEXRAD products are divided into multiple data processing levels.
The lower Level II data contain the three meteorological base data quantities at original resolution: reflectivity, mean radial velocity, and spectrum width.
With the advent of dual polarization beginning in 2011, additional base products of differential reflectivity, correlation coefficient and differential phase are available.
Mdm files are provided at the top of the hour. I believe these are summary files of the last hour. 

In [ ]:
%%time
# Tries to read in already present scan data. If the data is not present, download the files

relevantFiles = {}

for ID in RELEVANT_RADAR_IDS:
    
    radarPath = PATH+ID+EVENT_DATE
    
    try:
        
        files = os.listdir(radarPath)
        files.sort()
        relevantFiles[ID] = [radarPath + '/' + file for file in files if 'MDM' not in file and '.' not in file and ID in file] #remove MDM and any other files
        
    except:
        
        conn = nexradaws.NexradAwsInterface()
        scans = conn.get_avail_scans_in_range(START, END, ID)

        with contextlib.redirect_stdout(None):
            results = conn.download(scans, radarPath) #Downloads all level 2 scans.
            
        files = os.listdir(radarPath)
        files.sort()
        relevantFiles[ID] = [radarPath + '/' + file for file in files if 'MDM' not in file]
        
    print("{}: There are {} scans available between {} and {}\n".format(ID, len(relevantFiles[ID]), START, END))

In [ ]:
#parameters:
#    angles -- a 2D list of angles where each list represents a set of scan angles.
#    DESIRED_ANGLES -- a global varaible representing the angles we want to generate QVPs for
#    THRESHOLD -- The minimum percentage of scans that an angle must be present in
#    RELATIVE_ERROR_TOLERANCE -- a global variable representing the acceptable relative error between our desired angle and the chosen angle
#Returns a list of the angles that are closest to the DESIRED_ANGLES. Angles must be present in at least THRESHOLD percent of the scans throughout the interval.
#It will choose the angle that has the lowest relative error that is below RELATIVE_ERROR_TOLERANCE.
#Appends None if no such angle exists

def find_closest_angles(angles, DESIRED_ANGLES, THRESHOLD, RELATIVE_ERROR_TOLERANCE):
    
    closestAngles = []
    
    for desiredAngle in DESIRED_ANGLES:
        
        argmins = []
        argvals = []


        for angleList in angles:

            argmins.append(abs(np.array(angleList) - desiredAngle).argmin())
            argvals.append(angleList[argmins[-1]])

        valCounts = Counter(argvals)
        uniqueVals = np.unique(argvals)
        n = len(angles)

        for val in uniqueVals:
            if valCounts[val]/n < THRESHOLD:
                uniqueVals = np.delete(uniqueVals,np.where(uniqueVals == val))

        if len(uniqueVals) == 0:
            return None

        relativeErrors = abs(desiredAngle-uniqueVals)/desiredAngle
        idx = relativeErrors.argmin()

        if relativeErrors[idx] > RELATIVE_ERROR_TOLERANCE:
            
            closestAngles.append(None)
            
        else:
            
            closestAngles.append(uniqueVals[idx])
    
    return closestAngles

#Returns a 2D list containing the angles of each scan over the interval for a given radar
def generate_angles(radarID,files):
    
    angles = []
    
    for file in files[radarID]:
        try:
            
            radar = pyart.io.read_nexrad_archive(file, include_fields = []) #not including the fields speeds us up .25s per file
            angles.append(radar.fixed_angle['data'].tolist())
            
        except:
            
            continue

    return angles

#Generates QVPs for a given scan (file) and desired angles
#Returns a list of QVP dictionaries consisting of level II data averaged at all heights given for each angle
def quasi_vertical_profile(filename, closestAngles, fields=None, gatefilter=None):
    
    qvps = []
    
    radar = pyart.io.read_nexrad_archive(filename)
    
    scanAngles = radar.fixed_angle['data']
    
    for closestAngle in closestAngles:
        
        if closestAngle not in scanAngles:
            
            qvps.append(None)
            
            continue
            
        qvp = {}
        
        index = np.where(scanAngles == closestAngle)[0][0]

        if fields is None: #Goes through each file and finds the fields (i.e. reflectivity, cross correlation coefficient, etc. present. )
            fields = radar.fields

        for field in fields:
            
            thisField = radar.get_field(index, field).mean(axis = 0) #for each variable (i.e. reflectivity, cross correlation coefficient) calculate the mean at all heights given.
            thisFieldStd = radar.get_field(index, field).std(axis = 0)
            #thisFieldMad = pd.DataFrame(radar.get_field(index, field)).mad(axis=0)
            thisFieldMad = robust.mad(radar.get_field(index, field),axis=0)
            
            qvp.update({field:thisField})
            qvp.update({field+'_std':thisFieldStd})
            qvp.update({field+'_mad':np.array(thisFieldStd)})

        qvp.update({'range': radar.range['data'], 'time': radar.time}) #Updates range gates distance away from the radar
        x,y,z = pyart.core.antenna_to_cartesian(qvp['range']/1000.0, 0.0, closestAngle) #Calculates range gates x,y, and height relative to distance away from the radar 
        qvp.update({'height': z})
        qvps.append(qvp)
    
    return qvps

In [ ]:
%%time
#We are going to loop through each file representative of a complete cycle of scans.  We will average the data and then 
#concatenate it into a pandas dataframe named after the individual variables
#Saves all scan data to radarPath including error logs, a .txt of the scan angles and QVP Figures, Gradients and Standard Deviations
#Will not generate QVP figures if the percentage of unusable scans from a radar exceeds THRESHOLD

scanAngles = {} #Allows us to save and inspect the scan angles for each radar's scans
unusableScans = 0

for ID in RELEVANT_RADAR_IDS:

    radarPath = PATH+ID+EVENT_DATE
    
    #see if angles have already been stored for this radar. If so, load them in. If not, generate them by reading the files. Saves a lot of time
    try:

        with open(radarPath + '/Available Angles','rb') as f:    
            scanAngles[ID] = pickle.load(f)

    except:

        scanAngles[ID] = generate_angles(ID,relevantFiles)

        with open(radarPath + '/Available Angles','wb') as f:
            pickle.dump(scanAngles[ID],f)


    closestAngles = find_closest_angles(scanAngles[ID],DESIRED_ANGLES,THRESHOLD,RELATIVE_ERROR_TOLERANCE) #find the closest angles for all the desired angles

    for idx, closestAngle in enumerate(closestAngles.copy()):

        if closestAngle == None:

            print('No suitable angle for ' + ID, 'at ' + str(DESIRED_ANGLES[idx]) + ' degrees.')
            closestAngles.remove(closestAngle)
            continue
            
            
        figurePath = radarPath + '/Figures/' + str(closestAngle)[:5]
        
        if os.path.exists(figurePath): #if figures for this angle already exists for this radar, we remove the angle to avoid redundancy
            
            closestAngles.remove(closestAngle)

        else: #Otherwise, we make the directory to place figures into
            
            os.makedirs(figurePath)
        
    if len(closestAngles) == 0: #if all the figures for this radar have been generated, we go onto the next radar
        
        continue

    all_qvps = [[] for i in range(len(closestAngles))] #we're going to append the qvps for each angle to the corresponding list

    for file_idx in range(len(relevantFiles[ID])):

        try:

            returned_qvps = quasi_vertical_profile(relevantFiles[ID][file_idx],closestAngles)

            for qvp_idx, qvp in enumerate(returned_qvps):


                if qvp == None:

                    continue

                all_qvps[qvp_idx].append(qvp)
        
        except:
            
            reload(logging) #ipython will override your logging handler so you must reload
            logging.basicConfig(filename=radarPath + '/Pyart File Read Errors.log', encoding='utf-8', level=logging.DEBUG) #at least one scan is unreadable

            debugString= ' filename='+relevantFiles[ID][idx] + ' index=' + str(idx) + ' at ' + str(datetime.now().time())[:-5]
            logging.debug(debugString)
            logging.exception(e)

            if 'index 0 is out of bounds for axis 0 with size 0' in str(e):     #this is the error thrown by unreadable files, so increment unusableScans
                unusableScans +=1

            #print(traceback.format_exc())

    for idx, qvp_list in enumerate(all_qvps):
                
        reflectivities = pd.DataFrame()
        reflectivities_std = pd.DataFrame()
        reflectivities_mad = pd.DataFrame()
        differential_phases = pd.DataFrame()
        differential_phases_std = pd.DataFrame()
        differential_phases_mad = pd.DataFrame()
        cross_correlation_ratios = pd.DataFrame()
        cross_correlation_ratios_std = pd.DataFrame()
        cross_correlation_ratios_mad = pd.DataFrame()
        spectrum_widths = pd.DataFrame()
        spectrum_widths_std = pd.DataFrame()
        spectrum_widths_mad = pd.DataFrame()
        differential_reflectivities = pd.DataFrame()
        differential_reflectivities_std = pd.DataFrame()
        differential_reflectivities_mad = pd.DataFrame()
        heights = pd.DataFrame()
        times = []
                
        for qvp in qvp_list:
            
            reflectivity = pd.DataFrame(qvp['reflectivity']).T
            differential_phase = pd.DataFrame(qvp['differential_phase']).T
            cross_correlation_ratio = pd.DataFrame(qvp['cross_correlation_ratio']).T
            spectrum_width = pd.DataFrame(qvp['spectrum_width']).T
            differential_reflectivity = pd.DataFrame(qvp['differential_reflectivity']).T
            height = pd.DataFrame(qvp['height']).T

            reflectivity_std = pd.DataFrame(qvp['reflectivity_std']).T
            differential_phase_std = pd.DataFrame(qvp['differential_phase_std']).T
            cross_correlation_ratio_std = pd.DataFrame(qvp['cross_correlation_ratio_std']).T
            spectrum_width_std = pd.DataFrame(qvp['spectrum_width_std']).T
            differential_reflectivity_std = pd.DataFrame(qvp['differential_reflectivity_std']).T
            
            reflectivity_mad = pd.DataFrame(qvp['reflectivity_mad']).T
            differential_phase_mad = pd.DataFrame(qvp['differential_phase_mad']).T
            cross_correlation_ratio_mad = pd.DataFrame(qvp['cross_correlation_ratio_mad']).T
            spectrum_width_mad = pd.DataFrame(qvp['spectrum_width_std']).T
            differential_reflectivity_mad = pd.DataFrame(qvp['differential_reflectivity_mad']).T

            reflectivities = pd.concat([reflectivities, reflectivity]) #concatenates qvp for reflectivity we just calculated with all of the other qvps such that we get a time series of all the qvps from each file
            differential_phases = pd.concat([differential_phases, differential_phase])
            cross_correlation_ratios = pd.concat([cross_correlation_ratios, cross_correlation_ratio])
            spectrum_widths= pd.concat([spectrum_widths, spectrum_width])
            differential_reflectivities = pd.concat([differential_reflectivities, differential_reflectivity])

            reflectivities_std = pd.concat([reflectivities_std, reflectivity_std])
            differential_phases_std = pd.concat([differential_phases_std, differential_phase_std])
            cross_correlation_ratios_std = pd.concat([cross_correlation_ratios_std, cross_correlation_ratio_std])
            spectrum_widths_std = pd.concat([spectrum_widths_std, spectrum_width_std])
            differential_reflectivities_std = pd.concat([differential_reflectivities_std, differential_reflectivity_std])
            
            reflectivities_mad = pd.concat([reflectivities_mad, reflectivity_mad])
            differential_phases_mad = pd.concat([differential_phases_mad, differential_phase_mad])
            cross_correlation_ratios_mad = pd.concat([cross_correlation_ratios_mad, cross_correlation_ratio_mad])
            spectrum_widths_mad = pd.concat([spectrum_widths_mad, spectrum_width_mad])
            differential_reflectivities_mad = pd.concat([differential_reflectivities_mad, differential_reflectivity_mad])


            heights = pd.concat([heights, height])
            times.append(qvp['time']['units'][14:-1])
                
        
        
        #In order to do a pcolormesh you need three equal sized arrays of time,height,and then the variable you want to plot.
        #Creates a 2d array of time that is the same size as our height and variable arrays/dataframes
        #creates pandas dataframe by repeating the times n number of times where n is the size of the height array in the y direction 

        times = pd.DataFrame(pd.to_datetime(times))
        n = reflectivities.shape[1]
        times_modified = pd.concat([times] * (n), axis=1, ignore_index=True) 
        new_reflectivities = reflectivities.copy(deep = True)

        df = new_reflectivities
        df.index = np.arange(0, df.shape[0], 1)
        s=df.isnull().stack()
        s=s.groupby(level=0).cumsum()[~s]
        s=s.groupby([s.index.get_level_values(0),s]).transform('count').unstack().reindex_like(df)


        reflectivities_deep = reflectivities.copy(deep = True)
        reflectivities_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        differential_phases_deep = differential_phases.copy(deep = True)
        differential_phases_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        cross_correlation_ratios_deep = cross_correlation_ratios.copy(deep = True)
        cross_correlation_ratios_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        differential_reflectivities_deep = differential_reflectivities.copy(deep = True)
        differential_reflectivities_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        reflectivities_std_deep = reflectivities_std.copy(deep = True)
        reflectivities_std_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        differential_phases_std_deep = differential_phases_std.copy(deep = True)
        differential_phases_std_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        cross_correlation_ratios_std_deep = cross_correlation_ratios_std.copy(deep = True)
        cross_correlation_ratios_std_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        differential_reflectivities_std_deep = differential_reflectivities_std.copy(deep = True)
        differential_reflectivities_std_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)
        
        reflectivities_mad_deep = reflectivities_mad.copy(deep = True)
        reflectivities_mad_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        differential_phases_mad_deep = differential_phases_mad.copy(deep = True)
        differential_phases_mad_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        cross_correlation_ratios_mad_deep = cross_correlation_ratios_mad.copy(deep = True)
        cross_correlation_ratios_mad_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)

        differential_reflectivities_mad_deep = differential_reflectivities_mad.copy(deep = True)
        differential_reflectivities_mad_deep.index = np.arange(0, reflectivities_deep.shape[0], 1)
        
        closestAngle = closestAngles[idx]
        figurePath = radarPath + '/Figures/' + str(closestAngle)[:5]


        #Reflectivity QVP
        ymin = 0
        ymax = 12

        cmap_reflectivity = mpl.colors.ListedColormap(['#e3fdfe','#d7d2e7','#cbb1d2','#9d80a4','#6d4b74','#d6d4b1','#a9a882','#777777','#6be9ea','#469eef','#0e00ec','#75fb4c','#5ac53a','#3f8e27','#feff54','#e1c140','#f09636','#ea3423','#ca2A1d','#b02318','#ea33f7','#8f59c3'])
        bounds_reflectivity = [-35, -30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]
        norm_reflectivity = mpl.colors.BoundaryNorm(bounds_reflectivity, cmap_reflectivity.N)

        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, reflectivities_deep[(s>5) & (reflectivities_deep > 0)].T, cmap=cmap_reflectivity, norm = norm_reflectivity)
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Reflectivity QVP '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Reflectivity QVP.png', dpi = 300)
        plt.close()

        #Reflectivity QVP Gradient
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, reflectivities_deep[(s>5) & (reflectivities_deep > 0)].T.diff(10, axis = 0) * -1, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Reflectivity QVP Gradient '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Reflectivity Gradient.png', dpi = 300)
        plt.close()

        #Reflectivity Standard Deviations
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, reflectivities_std_deep[(s>5) & (reflectivities_std_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Reflectivity Standard Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Reflectivity Standard Deviations.png', dpi = 300)
        plt.close()
        
        #Reflectivity MAD
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, reflectivities_mad_deep[(s>5) & (reflectivities_mad_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Reflectivity Mean Absolute Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Reflectivity Mean Absolute Deviations.png', dpi = 300)
        plt.close()

        #Differential Phase QVP
        cmap_kdp = mpl.colors.ListedColormap(['#a5a5a5','#7f7f7f','#6f1812','#a62a1f','#e73755','#ea95ba','#c694e7','#7d41bd','#5623bc','#73f6fc','#5eb5de','#2f6fba','#4a9b62','#6fcc78','#80f6af','#79fb54','#feff54','#f9da78','#df8344','#ffffff'])
        bounds_kdp = [-0.3, -0.1, 0, 0.02, 0.04, 0.07, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 2, 10, 30]
        norm_kdp = mpl.colors.BoundaryNorm(bounds_kdp, cmap_kdp.N)

        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_phases_deep[s > 5].T, cmap=cmap_kdp, norm = norm_kdp)
        plt.ylabel('Height (km ASL)', fontsize = 14)
        plt.xlabel('Time (UTC)', fontsize = 14)
        plt.title(ID + ' Differential Phase QVP ' + str(closestAngle)[:-5] + ' Degrees' + EVENT_DATE, fontsize = 18)
        plt.colorbar(cs, pad=0.05)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Phase QVP.png', dpi = 300)
        plt.close()

        #Differential Phase Gradient
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_phases_deep[(s>5) & (differential_phases_deep > 0)].T.diff(10, axis = 0) * -1, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Differential Phase Gradient '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Phase Gradient.png', dpi = 300)
        plt.close()

        #Differential Phase Standard Deviations
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_phases_std_deep[(s>5) & (differential_phases_std_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Differential Phase Standard Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Phase Standard Deviations.png', dpi = 300)
        plt.close()
        
        #Differential Phase MAD
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_phases_mad_deep[(s>5) & (differential_phases_mad_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Differential Phase Mean Absolute Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Phase Mean Absolute Deviations.png', dpi = 300)
        plt.close()

        #Cross Correlation Coefficient QVP
        cmap_cc = mpl.colors.ListedColormap(['#95949b','#151486','#0d01e0','#8987d1','#8dfc71','#9acd3f','#fefb53','#f5c644','#df8f35','#ea4425','#d12c1e','#941c12','#6f1357','#efb0cf','#6d1379'])
        bounds_cc = [0.2, 0.45, 0.65, 0.75, 0.8, 0.85, 0.90, 0.93, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00, 1.05, 3]
        norm_cc = mpl.colors.BoundaryNorm(bounds_cc, cmap_cc.N)

        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, cross_correlation_ratios_deep[(s>5) & (reflectivities_deep > 0)].T, norm = norm_cc, cmap = cmap_cc)
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Cross Correlation Coefficient QVP ' + str(closestAngle)[:-5] + ' Degrees' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Cross Correlation QVP.png', dpi = 300)
        plt.close()

        #Cross Correlation Coefficient Gradient
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, cross_correlation_ratios_deep[(s>5) &
                                                                                                              (cross_correlation_ratios_deep > 0)].T.diff(10, axis = 0) * -1, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Cross Correlation Coefficient Gradient '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Cross Correlation Gradient.png', dpi = 300)
        plt.close()

        #Cross Correlation Coefficient Standard Deviations
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, cross_correlation_ratios_std_deep[(s>5) & (cross_correlation_ratios_std_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Cross Correlation Coefficient Standard Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Cross Correlation Coefficient Standard Deviations.png', dpi = 300)
        plt.close()
        
        #Cross Correlation Coefficient MAD
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, cross_correlation_ratios_mad_deep[(s>5) & (cross_correlation_ratios_mad_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Cross Correlation Coefficient Mean Absolute Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Cross Correlation Coefficient Mean Absolute Deviations.png', dpi = 300)
        plt.close()


        #Differential Reflectivity QVP
        cmap_zdr = mpl.colors.ListedColormap(['#404040','#9c9c9c','#c9c9c9','#8879b0','#050092','#4B96ce','#83fbd4','#7dd868','#ffff7a','#F09655','#c82a1d','#9f1f14','#e888bc','#ffffff','#6d1379'])
        bounds_zdr = [-4, -2, -0.5, 0, 0.3, 0.6, 1, 1.5, 2, 2.5, 3, 4, 5, 6, 8, 20]
        norm_zdr = mpl.colors.BoundaryNorm(bounds_zdr, cmap_zdr.N)

        fig, ax = plt.subplots(figsize = (15, 5))
        cs = ax.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_reflectivities_deep[(s>5) & (reflectivities_deep > 0)].T, cmap=cmap_zdr, norm = norm_zdr)
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Differential Reflectivity QVP ' + str(closestAngle)[:-5] + ' Degrees' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Reflectivity QVP.png', dpi = 300)
        plt.close()

        #Differential Reflectivity Gradient
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_reflectivities_deep[(s>5) &
                                                                                                                 (differential_reflectivities_deep > 0)].T.diff(10, axis = 0) * -1, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Differential Reflectivity Gradient '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Reflectivity Gradient.png', dpi = 300)
        plt.close()

        #Differential Reflectivity Standard Deviations
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_reflectivities_std_deep[(s>5) & (differential_reflectivities_std_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Differential Reflectivity Standard Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Reflectivity Standard Deviations.png', dpi = 300)
        plt.close()
        
        #Differential Reflectivity MAD
        plt.figure(figsize = (15, 5))
        cs = plt.pcolormesh(times_modified.loc[:, 0], heights.loc[0, :].T/1000, differential_reflectivities_mad_deep[(s>5) & (differential_reflectivities_mad_deep > 0)].T, cmap='seismic')
        plt.ylabel('Height (km ASL)', fontsize = 20)
        plt.xlabel('Time (UTC)', fontsize = 20)
        plt.title(ID + ' Differential Reflectivity Mean Absolute Deviations '+ str(closestAngle)[:-5] + ' Degrees ' + EVENT_DATE, fontsize = 18)
        cbar = plt.colorbar(cs, pad=0.005)
        cbar.ax.tick_params(labelsize = 16)
        xformatter = md.DateFormatter('%H:%M')
        xlocator = md.MinuteLocator(byminute=[0])
        plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
        plt.tick_params(labelsize = 18)
        plt.ylim(ymin, ymax)
        plt.savefig(figurePath + '/Differential Reflectivity Mean Absolute Deviations.png', dpi = 300)
        plt.close()

        print(ID + ' ' + str(closestAngle) + ' degrees: Done!')

print('Finished!')
